In [1]:
%load_ext autoreload
%autoreload 2

### Data Cleaning

In [2]:
import pandas as pd


df = pd.read_csv("./data-june25-jul23.csv")

print(df.columns.tolist())

df.head()


['Date', 'Team Start', 'Team 1 Player 1 ', 'Team 1 Player 2 ', 'Team 2 Player 1 ', 'Team 2 Player 2 ', 'Team 1 Score ', 'Team 2 Score', 'Winning Team']


,Date,Team Start,Team 1 Player 1,Team 1 Player 2,Team 2 Player 1,Team 2 Player 2,Team 1 Score,Team 2 Score,Winning Team
0,6/25/25,Team 1,Erick A,Miles W,Mathis W,Nico U,19.0,17.0,Team 1
1,6/25/25,Team 2,Angel C,Anthony C,Erick A,Miles W,12.0,15.0,Team 2
2,6/25/25,Team 1,Erick A,Miles W,Mathis W,Nico U,17.0,15.0,Team 1
3,6/25/25,Team 1,Angel C,Miles W,Anthony C,Nico U,9.0,15.0,Team 2
4,6/25/25,Team 1,Angel C,Anthony C,Mathis W,Nico U,11.0,15.0,Team 2


In [3]:
player_columns = ['Team 1 Player 1 ', 'Team 1 Player 2 ', 'Team 2 Player 1 ', 'Team 2 Player 2 ']
players = pd.unique(df[player_columns].values.ravel())

print(len(players))
players

18


array(['Erick A', 'Miles W', 'Mathis W', 'Nico U ', 'Angel C',
       'Anthony C', 'Nico U', 'Angel', 'Anthony', 'Erick', 'Nico',
       'Mathis', 'Noa', 'Miles', 'Daniel', 'Evan', 'Madison', 'Sarah'],
      dtype=object)

In [4]:
# Consolidate duplicate players (for now remove last name)
for col in player_columns:
    df[col] = df[col].str.split().str[0]

players = pd.unique(df[player_columns].values.ravel())

print(len(players))
players

11


array(['Erick', 'Miles', 'Mathis', 'Nico', 'Angel', 'Anthony', 'Noa',
       'Daniel', 'Evan', 'Madison', 'Sarah'], dtype=object)

In [5]:
elo_scores: dict[str, float] = {player: 1000 for player in players}
elo_scores

{'Erick': 1000,
 'Miles': 1000,
 'Mathis': 1000,
 'Nico': 1000,
 'Angel': 1000,
 'Anthony': 1000,
 'Noa': 1000,
 'Daniel': 1000,
 'Evan': 1000,
 'Madison': 1000,
 'Sarah': 1000}

### Basic ELO 
_Updating both players at the same time_

In [6]:
elo_scores: dict[str, float] = {player: 1000 for player in players}

# Initialize ELO history with original scores
elo_history: dict[str, list[dict[str, float]]] = {}
elo_history["original"] = [dict(elo_scores)]  # Make original a list too for consistency

def expected_score_for_r1(R1, R2):
    exp = (R2 - R1)/400
    return 1/(1 + 10**exp)

# K set to 32 like chess
def calc_new_rating(old_rating, actual, expected):
    return old_rating + (32 * (actual - expected))

team1_columns = ['Team 1 Player 1 ', 'Team 1 Player 2 ']
team2_columns = ['Team 2 Player 1 ', 'Team 2 Player 2 ']

# Process each game
for row in df.to_dict(orient="records"):
    date = row["Date"]
    winning_team = row["Winning Team"]
    
    if winning_team == "Team 1":
        winning_players = [row[col] for col in team1_columns]
        losing_players = [row[col] for col in team2_columns]
    else:
        winning_players = [row[col] for col in team2_columns]
        losing_players = [row[col] for col in team1_columns]
    
    # Get averaged team elo
    winning_elo = (sum(elo_scores[player] for player in winning_players if player in elo_scores) / 2)
    losing_elo = (sum(elo_scores[player] for player in losing_players if player in elo_scores) / 2)
    
    # Calculate expected scores (total % adds to 1)
    winning_expected = expected_score_for_r1(winning_elo, losing_elo)
    losing_expected = 1 - winning_expected
    
    # Calculate new ratings (1 is a win, 0 is a loss)
    new_winning_elo = calc_new_rating(winning_elo, 1, winning_expected)
    new_losing_elo = calc_new_rating(losing_elo, 0, losing_expected)
    
    # Update ELO scores with rounding
    for player in winning_players:
        elo_scores[player] = round(new_winning_elo, 1)
    for player in losing_players:
        elo_scores[player] = round(new_losing_elo, 1)
    
    # Store current ELO state in history
    if date not in elo_history:
        elo_history[date] = []
    
    # Store a snapshot of current ELO scores after this game
    elo_history[date].append(dict(elo_scores))


# Calculate total games played
total_games = sum(len(scores_list) for date_key, scores_list in elo_history.items() if date_key != "original")

print(f"Total games played: {total_games}")
print("\nFinal ELO Rankings:")

# Sort players by final ELO score (highest to lowest)
sorted_players = sorted(elo_scores.items(), key=lambda x: x[1], reverse=True)

for rank, (player, elo) in enumerate(sorted_players, 1):
    print(f"{rank}. {player}: {elo}")


Total games played: 31

Final ELO Rankings:
1. Erick: 1070.6
2. Sarah: 1040.6
3. Nico: 1018.4
4. Angel: 1018.4
5. Mathis: 1009.1
6. Anthony: 1009.1
7. Evan: 1007.2
8. Daniel: 977.5
9. Madison: 960.9
10. Noa: 946.8
11. Miles: 941.5


In [24]:
import plotly.graph_objects as go

# Prepare data for plotting
x_positions = []
x_labels = []
all_player_data = {player: [] for player in players}
hover_data = []  # Store custom hover info for each x position
x_pos = 0

# Add original scores (first entry in the "original" list)
original_scores = elo_history["original"][0]
for player in players:
    all_player_data[player].append(original_scores[player])

x_positions.append(x_pos)
x_labels.append("Start")
# For the start position, no changes to show
hover_data.append("Start - No changes yet")
x_pos += 1

# Process each date
for date_key in sorted(elo_history.keys()):
    if date_key == "original":
        continue
    
    games_on_date = elo_history[date_key]
    
    # Add each game for this date
    for game_idx, game_scores in enumerate(games_on_date):
        # Calculate changes from previous game
        changes = []
        previous_scores = {}
        
        # Get previous scores (from the last entry in all_player_data)
        for player in players:
            previous_scores[player] = all_player_data[player][-1]
        
        # Add current game scores and calculate changes
        for player in players:
            current_score = game_scores[player]
            previous_score = previous_scores[player]
            change = current_score - previous_score
            
            all_player_data[player].append(current_score)
            
            # Only include players who changed
            if change != 0:
                if change > 0:
                    triangle = "▲"  # Green up triangle
                    color_start = "<span style='color: green;'>"
                else:
                    triangle = "▼"  # Red down triangle
                    color_start = "<span style='color: red;'>"
                
                changes.append(f"{color_start}{player} {triangle} {change:+.1f}</span>")
        
        x_positions.append(x_pos)
        
        # Create hover text for this game
        if changes:
            hover_text = f"<b>{date_key} - Game {game_idx + 1}</b><br>" + "<br>".join(changes)
        else:
            hover_text = f"<b>{date_key} - Game {game_idx + 1}</b><br>No ELO changes"
        
        hover_data.append(hover_text)
        x_pos += 1
    
    # Add date label at the middle of games for this date
    if len(games_on_date) > 1:
        middle_pos = x_pos - len(games_on_date) + (len(games_on_date) - 1) / 2
    else:
        middle_pos = x_pos - 1
    x_labels.append((middle_pos, date_key))

# Create the figure
fig = go.Figure()

# Add a trace for each player
for i, player in enumerate(players):
    fig.add_trace(go.Scatter(
        x=x_positions,
        y=all_player_data[player],
        mode='lines+markers',
        name=player,
        line=dict(width=2),
        marker=dict(size=6),
        hoverinfo='skip', # start w/ these traces disabled
        hovertemplate=None,
        showlegend=True
    ))

# Add an invisible trace for the custom hover info 
fig.add_trace(go.Scatter(
    x=x_positions,
    y=[1000] * len(x_positions), 
    mode='markers',
    marker=dict(size=10, opacity=0), 
    hovertemplate='%{customdata}<extra></extra>',
    customdata=hover_data,
    showlegend=False,
    name='Game Info',
    visible=True,  # Start with game changes visible
    hoverinfo='text',
    text=hover_data  # Use text instead of customdata for better control
))

# Create custom x-axis labels
x_tick_positions = [0] + [pos for pos, label in x_labels[1:]]
x_tick_labels = ["Start"] + [label for pos, label in x_labels[1:]]

# Update layout
fig.update_layout(
    title={
        'text': 'ELO Rating Progression Over Time<br><sub>Click legend entries to show/hide players • Double-click to isolate</sub> <br><sub>Click on Game/Player buttons to change what hover shows (starts in Game)</sub><br>',
        'x': 0.5,
        'font': {'size': 16},
        'pad': {'b': 20} 
    },
    xaxis_title='Date / Games',
    yaxis_title='ELO Rating',
    width=1100,
    height=700,
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    margin=dict(r=150,t=130),  # Make room for legend, add some room for title
    updatemenus=[
        dict(
            active=0,
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{
                        "hoverinfo": ["skip"] * len(players) + ["text"],  # Disable player hovers, enable invisible trace
                        "hovertemplate": [None] * len(players) + ['%{customdata}<extra></extra>'],
                        "visible": [True] * len(players) + [True]  # Show invisible trace
                    }, {
                        "hovermode": "x unified"  # Use closest to target our invisible trace
                    }],
                    label="Game",
                    method="update"
                ),
                dict(
                    args=[{
                        "hoverinfo": ["x+y+name"] * len(players) + ["skip"],  # Enable player hovers, disable invisible trace
                        "visible": [True] * len(players) + [False]  # Hide invisible trace
                    }, {
                        "hovermode": "x unified"  # Back to unified mode for player scores
                    }],
                    label="Player",
                    method="update"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Set custom x-axis ticks
fig.update_xaxes(
    tickmode='array',
    tickvals=x_tick_positions,
    ticktext=x_tick_labels,
    tickangle=45
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()

In [ ]:
# Export to html
fig.write_html("index.html")